In [1]:
import gc
import os
import talib
import requests
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime, timezone, time


import plotly as py
import plotly.io as pio
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

from os import getenv
from dotenv import load_dotenv

from strategy import long_short, create_signal
from config import ccxt_connect, prices_collection, binance_client
from helper import vwap, current_position

load_dotenv()

client = ccxt_connect()
binance = binance_client()
prices = prices_collection()

In [22]:
df = pd.DataFrame(list(prices.find().sort('date', -1)))
df.sort_values('date', inplace=True)

df.set_index('date', inplace=True)
close = df['close'].values
df['rsi'] = talib.RSI(close, timeperiod=14)
macd, macdsignal, df['macdhist'] = talib.MACD(close, fastperiod=12, slowperiod=26, signalperiod=9)
df['EMA9'] = talib.EMA(df.close, 9)
df['EMA55'] = talib.EMA(df.close, 55)
df['EMA200'] = talib.EMA(df.close, 200)

df['date'] = df.index
df.reset_index(drop=True, inplace=True)
df.fillna(method='ffill', inplace=True)

df.drop_duplicates(subset='date', keep='last', inplace=True)

print('Calculating signals')
df['cross'] = df.apply(long_short, axis=1)

df = df[50000:]
df['signal'] = df[df['cross'] != 0]['cross'].diff()
df['signal'].fillna(0, inplace=True)

df['action'] = df.apply(create_signal, axis=1)

Calculating signals


In [25]:
def backtest(df, initial_capital=1000, taker_fee=0.0006, leverage=5):
  # Initialize the portfolio with an initial capital of 100
  portfolio = initial_capital
  
  # Initialize the position to None
  position = None

  # Set the taker fee to 0.06%
  taker_fee = taker_fee

  # Set the leverage to 5x
  leverage = leverage
  
  # Iterate through each row in the DataFrame
  for i, row in df.iterrows():
    # Check the value of the signal column
    if row['action'] == 'Buy':
      # If the signal is 1 and the position is None, buy the asset at the current price
      if position is None:
        investment = (portfolio * leverage) * (1 - taker_fee)
        # Update the portfolio value
        portfolio += investment * (row['close'] / investment)
        position = 'long'
      # If the signal is 1 and the position is short, close the short position and buy the asset at the current price
      elif position == 'short':
        investment = (portfolio * leverage) * (1 - taker_fee)
        # Update the portfolio value
        portfolio += 2 * investment * (row['close'] / investment)
        position = 'long'
    elif row['action'] == 'Sell':
      # If the signal is -1 and the position is None, short the asset at the current price
      if position is None:
        investment = (portfolio * leverage) * (1 - taker_fee)
        # Update the portfolio value
        portfolio -= investment * (row['close'] / investment)
        position = 'short'
      # If the signal is -1 and the position is long, sell the asset at the current price and short the asset at the current price
      elif position == 'long':
        investment = (portfolio * leverage) * (1 - taker_fee)
        # Update the portfolio value
        portfolio -= 2 * investment * (row['close'] / investment)
        position = 'short'
      
  # Return the final value of the portfolio
  return portfolio

In [26]:
backtest(df)

4271.370000000009

In [27]:
df.tail(40)

,_id,open,high,low,close,vol,rsi,macdhist,EMA9,EMA55,EMA200,date,cross,signal,action
572258,63ac9ce7ddd647035a089482,1194.11,1194.32,1192.00,1192.03,10.20351,27.982412,-0.128893,1192.257683,1192.674179,1194.547852,2022-12-28 16:49:59.999,1,-1.0,Sell
572266,63ac9a695d4521f3f4a94419,1192.13,1193.71,1192.13,1193.39,13.55543,64.347982,0.100657,1193.200029,1192.854881,1194.458831,2022-12-28 16:54:59.999,2,1.0,Buy
572273,63ac9e14ddd647035a089677,1193.03,1193.22,1192.67,1193.22,3.99445,57.752997,0.003459,1193.215812,1192.936943,1194.375078,2022-12-28 16:59:59.999,2,0.0,None
572280,63ac9a8eddd647035a08909f,1192.91,1193.66,1191.11,1192.00,68.08266,25.833178,-0.130291,1192.254974,1192.726362,1194.214507,2022-12-28 17:04:59.999,1,-1.0,Sell
572287,63ac9e14ddd647035a089679,1192.33,1193.23,1192.01,1193.20,8.72602,61.235608,0.095158,1193.001814,1192.832814,1194.145919,2022-12-28 17:09:59.999,2,1.0,Buy
572294,63ac9e14ddd647035a08967a,1192.97,1193.27,1191.89,1191.89,6.05965,32.652036,-0.091403,1192.123164,1192.620913,1193.993404,2022-12-28 17:14:59.999,1,-1.0,Sell
572301,63ac9a8eddd647035a0890a2,1191.87,1193.60,1191.86,1193.52,18.94159,65.912051,0.144701,1193.227062,1192.822986,1193.961399,2022-12-28 17:19:59.999,2,1.0,Buy
572308,63aca197ddd647035a089c55,1193.47,1193.98,1193.47,1193.77,2.97629,69.759965,0.053563,1193.656138,1193.035830,1193.948459,2022-12-28 17:24:59.999,2,0.0,None
572315,63ac9a8eddd647035a0890a4,1194.34,1194.76,1193.54,1194.76,3.75688,82.729381,0.068819,1194.528503,1193.423342,1194.003325,2022-12-28 17:29:59.999,2,0.0,None
572322,63ac9e14ddd647035a08967e,1194.57,1196.46,1194.57,1196.32,23.56560,91.911950,0.105565,1195.944296,1194.074375,1194.159947,2022-12-28 17:34:59.999,2,0.0,None


In [20]:
df = pd.DataFrame(list(prices.find().sort('date', -1)))
df.sort_values('date', inplace=True)

df.set_index('date', inplace=True)
close = df['close'].values
df['rsi'] = talib.RSI(close, timeperiod=14)
df['BB_up'], df['BB_mid'], df['BB_low'] = talib.BBANDS(close, timeperiod=20, nbdevup=2.3, nbdevdn=2.3, matype=0)
macd, macdsignal, df['macdhist'] = talib.MACD(close, fastperiod=12, slowperiod=26, signalperiod=9)
df['atr'] = talib.ATR(df['high'].values, df['low'].values, close, timeperiod=14)
df['obv'] = talib.OBV(close, df['vol'].values)
df['EMA9'] = talib.EMA(df.close, 9)
df['EMA55'] = talib.EMA(df.close, 55)
df['EMA200'] = talib.EMA(df.close, 200)

df['date'] = df.index
df.reset_index(drop=True, inplace=True)
df.fillna(method='ffill', inplace=True)

df.drop_duplicates(subset='date', keep='last', inplace=True)

print('Calculating signals')
df['cross'] = df.apply(long_short, axis=1)


df['Prev_High'] = df['high'].shift(1)
df['Prev_Low'] = df['low'].shift(1)
df['Prev_Close'] = df['close'].shift(1)

df['OO'] = df['open']-df['open'].shift(1)
df['OC'] = df['open']-df['Prev_Close']

df['Ret'] = (df['open'].shift(-1)-df['open'])/df['open']

t = .8
split = int(t*len(df))
split

# Create a column by name, 'Signal' and initialize with 0
df['signal'] = 0

# Assign a value of 1 to 'Signal' column for the quantile with the highest returns
df.loc[df['Ret'] > df['Ret'][:split].quantile(q=0.66), 'signal'] = 1

# Assign a value of -1 to 'Signal' column for the quantile with the lowest returns
df.loc[df['Ret'] < df['Ret'][:split].quantile(q=0.34), 'signal'] = -1

df = df[500:]
y = df['signal']
X = df.drop(['close', 'signal', 'high', 'low', 'vol', 'Ret', '_id'], axis=1)

Calculating signals


In [22]:
X

,open,rsi,BB_up,BB_mid,BB_low,macdhist,atr,obv,EMA9,EMA55,EMA200,date,cross,Prev_High,Prev_Low,Prev_Close,OO,OC
500,169.53,64.563642,169.989747,168.4500,166.910253,0.264431,0.153151,-2.625005e+02,169.124650,169.401652,176.773693,2019-09-25 06:34:59.999,0,169.53,169.53,169.53,0.00,0.00
501,168.98,50.381321,170.038409,168.5025,166.966591,0.207850,0.181497,-2.635005e+02,169.095720,169.386593,176.696144,2019-09-25 06:39:59.999,0,169.53,169.53,169.53,-0.55,-0.55
502,168.98,50.381321,170.077515,168.5550,167.032485,0.162998,0.168533,-2.635005e+02,169.072576,169.372072,176.619366,2019-09-25 06:44:59.999,0,168.98,168.98,168.98,0.00,0.00
503,168.79,41.217288,170.070522,168.5835,167.096478,0.096845,0.190781,-2.655005e+02,168.958061,169.340926,176.538577,2019-09-25 06:49:59.999,0,168.98,168.98,168.98,-0.19,-0.19
504,168.50,41.217288,170.054853,168.6150,167.175147,0.051220,0.177153,-2.655005e+02,168.866449,169.310893,176.458591,2019-09-25 06:54:59.999,0,168.79,168.50,168.50,-0.29,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
568509,1196.53,58.819621,1196.801124,1196.3475,1195.893876,0.020652,0.583685,-4.148705e+07,1196.398906,1196.330272,1196.174830,2022-12-28 11:39:59.999,2,1196.38,1196.27,1196.27,0.22,0.26
568513,1196.04,66.634778,1196.726422,1196.3655,1196.004578,0.030900,0.621219,-4.148704e+07,1196.523536,1196.368143,1196.191893,2022-12-28 11:44:59.999,2,1196.66,1196.46,1196.46,-0.49,-0.42
568516,1197.16,72.292845,1196.866669,1196.4465,1196.026331,0.033592,0.646891,-4.148703e+07,1196.624290,1196.405545,1196.207797,2022-12-28 11:49:59.999,2,1196.61,1195.88,1196.61,1.12,0.55
568518,1196.57,28.914162,1196.969054,1196.4155,1195.861946,-0.060104,0.680381,-4.148703e+07,1196.356346,1196.368676,1196.201306,2022-12-28 11:54:59.999,1,1197.20,1196.45,1196.73,-0.59,-0.16


In [17]:
def backtest(df):
  # Initialize the portfolio with an initial capital of 100
  portfolio = 100
  
  # Iterate through each row in the DataFrame
  for i, row in df.iterrows():
    # Check the value of the signal column
    if row['cross'] == 1:
      # If the signal is 1, buy the asset at the current price
      portfolio += portfolio * (row['close'] / portfolio)
    elif row['cross'] == 2:
      # If the signal is -1, sell the asset at the current price
      portfolio -= portfolio * (row['close'] / portfolio)
      
  # Return the final value of the portfolio
  return portfolio

In [18]:
backtest(df)

-11461008.299999978

In [38]:
print('Starting the bot')
print(f"Current time: {datetime.utcnow().strftime('%m-%d-%y-%H:%M:%S')}")

last_date = prices.find().sort('date', -1).limit(1).next()['date']
print(f"Finding the last record in our DB: \n{last_date.strftime('%m-%d-%y-%H:%M:%S')}")

print(f'Querying data from bybit to make current')
new_data_query = binance.get_klines(symbol='ETHUSD', interval=binance.KLINE_INTERVAL_5MINUTE, endTime=int(last_date.timestamp() * 1000))
print('Getting new data from Bybit API (5m interval)')

new_data = pd.DataFrame(new_data_query[:-1], columns=['Open time', 'open', 'high', 'low', 'close', 'vol', 'Close time', 'Quote asset volume', 'Number of trades', 'Taker buy base asset volume', 'Taker buy quote asset volume', 'Ignore'])
new_data['date'] = pd.to_datetime(new_data['Close time'], unit='ms')
cols_to_drop = ['Open time', 'Close time', 'Quote asset volume', 'Number of trades', 'Taker buy base asset volume', 'Taker buy quote asset volume', 'Ignore']
new_data.drop(cols_to_drop, axis=1, inplace=True)

cols = ['open', 'high', 'low', 'close', 'vol']
new_data[cols] = new_data[cols].apply(pd.to_numeric, errors='ignore', axis=1)

if len(new_data) > 0:
    print(f'Inserting new data into DB')
    prices_collection().insert_many(new_data.to_dict('records'))

df = pd.DataFrame(list(prices.find().sort('date', -1).limit(300)))
df.sort_values('date', inplace=True)

df.set_index('date', inplace=True)
close = df['close'].values
df['rsi'] = talib.RSI(close, timeperiod=14)
macd, macdsignal, df['macdhist'] = talib.MACD(close, fastperiod=12, slowperiod=26, signalperiod=9)
df['EMA9'] = talib.EMA(df.close, 9)
df['EMA55'] = talib.EMA(df.close, 55)
df['EMA200'] = talib.EMA(df.close, 200)

df['date'] = df.index
df.reset_index(drop=True, inplace=True)
df.fillna(method='ffill', inplace=True)

print('Calculating signals')
df['signal'] = df.apply(long_short, axis=1)

symbol = 'ETHUSD'

signal = df['signal'].iloc[-1]
price = df['close'].iloc[-1]
position = current_position()
print(f'Current signal: {signal}, current position: {position}')

Starting the bot
Current time: 12-26-22-19:29:49
Finding the last record in our DB: 
12-26-22-17:14:59
Querying data from bybit to make current
Getting new data from Bybit API (5m interval)
Inserting new data into DB
Calculating signals
Current signal: BUY, current position: SELL


In [39]:
df.tail(40)

,_id,open,high,low,close,vol,rsi,macdhist,EMA9,EMA55,EMA200,date,signal
260,63a9cfd28b7245c363620482,1216.49,1216.99,1216.49,1216.99,3.56191,60.460554,0.056139,1216.723856,1216.147237,1217.815917,2022-12-26 16:44:59.999,BUY
261,63a9d0fd8b7245c363620484,1216.86,1217.05,1216.60,1216.60,1.87930,54.654291,0.021375,1216.699085,1216.163407,1217.803819,2022-12-26 16:49:59.999,BUY
262,63a9f6307f44f8a85be4b9ec,1216.88,1217.19,1216.46,1216.62,58.39241,54.893519,-0.003248,1216.683268,1216.179714,1217.792039,2022-12-26 16:49:59.999,None
263,63a9d21028cdadb9e09ab0fa,1216.08,1216.27,1215.92,1215.92,2.22594,45.788400,-0.066166,1216.530614,1216.170439,1217.773412,2022-12-26 16:54:59.999,None
264,63a9f6307f44f8a85be4b9ed,1216.46,1216.64,1215.65,1216.20,40.69693,49.403573,-0.087095,1216.464491,1216.171494,1217.757756,2022-12-26 16:54:59.999,None
265,63a9d34acde3c6672b420661,1215.97,1216.85,1215.97,1216.63,6.33693,54.429496,-0.070839,1216.497593,1216.187870,1217.746535,2022-12-26 16:59:59.999,None
266,63a9f6307f44f8a85be4b9ee,1216.00,1217.07,1216.00,1216.54,60.09983,53.237506,-0.065151,1216.506075,1216.200446,1217.734529,2022-12-26 16:59:59.999,None
267,63a9d7717f44f8a85be4b813,1216.00,1217.07,1216.00,1216.54,60.09983,53.237506,-0.060494,1216.512860,1216.212573,1217.722644,2022-12-26 16:59:59.999,None
268,63a9f6307f44f8a85be4b9ef,1216.49,1216.49,1214.40,1214.40,123.22895,33.192141,-0.193158,1216.090288,1216.147838,1217.689582,2022-12-26 17:04:59.999,SELL
269,63a9d7717f44f8a85be4b814,1216.49,1216.49,1214.40,1214.40,123.22895,33.192141,-0.267929,1215.752230,1216.085415,1217.656850,2022-12-26 17:04:59.999,SELL


In [40]:
orders = client.fetch_closed_orders(symbol)

In [41]:
orders[-1]['info']['side']

'Sell'

In [42]:
int(orders[-1]['info']['qty'])

2681

In [44]:
signal = 'BUY'

In [45]:
if signal == 'BUY':
    print(f'Signal went from: {position} to {signal}, flipping to long at {price}')
    balance = client.fetch_balance()
    last_order = client.fetch_closed_orders(symbol)
    if last_order[-1]['info']['side'] == 'Sell':
        close = client.create_order(
                symbol = 'ETHUSD',
                type = 'market',
                side = 'buy',
                amount = int(last_order[-1]['info']['qty']),
                params = {'reduce_only': True})

    quantity = round((float(balance['ETH']['free']) * .95), 3)
    order = client.create_order(
            symbol = 'ETHUSD',
            type = 'market',
            side = 'buy',
            amount = (price * quantity) * 5,
            params = {'leverage': 5})
    print(order)

Signal went from: SELL to BUY, flipping to long at 1218.11
{'info': {'user_id': '48899600', 'order_id': '723dacdd-82ee-4bfb-8b9d-89428ea52bae', 'symbol': 'ETHUSD', 'side': 'Buy', 'order_type': 'Market', 'price': '1269.35', 'qty': '5536', 'time_in_force': 'ImmediateOrCancel', 'order_status': 'Created', 'last_exec_time': '0', 'last_exec_price': '0', 'leaves_qty': '5536', 'cum_exec_qty': '0', 'cum_exec_value': '0', 'cum_exec_fee': '0', 'reject_reason': 'EC_NoError', 'order_link_id': '', 'created_at': '2022-12-27T14:45:44.639Z', 'updated_at': '2022-12-27T14:45:44.639Z', 'take_profit': '0.00', 'stop_loss': '0.00', 'tp_trigger_by': 'UNKNOWN', 'sl_trigger_by': 'UNKNOWN'}, 'id': '723dacdd-82ee-4bfb-8b9d-89428ea52bae', 'clientOrderId': None, 'timestamp': 1672152344639, 'datetime': '2022-12-27T14:45:44.639Z', 'lastTradeTimestamp': None, 'symbol': 'ETH/USD:ETH', 'type': 'market', 'timeInForce': 'IOC', 'postOnly': False, 'side': 'buy', 'price': 1269.35, 'triggerPrice': None, 'stopPrice': None, 'am

In [47]:
order['info']['updated_at'], order['info']['symbol']

'ETHUSD'

In [ ]:
if signal == position or signal == None:
        print('No change in position, sleep for 5min and rescan')

else:
        with open('current_position.txt', 'w') as f:
                f.write(str(signal))

if signal == 'BUY':
        print(f'Signal went from: {position} to {signal}, flipping to long at {price}')
        balance = client.fetch_balance()
        close = client.create_order(
                symbol = 'ETHUSD',
                type = 'market',
                side = 'sell',
                amount = (price * balance['ETH']['used']) * 5,
                params = {'reduce_only': True})
        quantity = round((float(balance['ETH']['free']) * .95), 3)
        order = client.create_order(
                symbol = 'ETHUSD',
                type = 'market',
                side = 'buy',
                amount = (price * quantity) * 5,
                params = {'leverage': 5})
        print(order)

if signal == 'SELL':
        print(f'Signal went from: {position} to {signal}, flipping to short at {price}')
        balance = client.fetch_balance()
        close = client.create_order(
                symbol = 'ETHUSD',
                type = 'market',
                side = 'buy',
                amount = (price * balance['ETH']['used']) * 5,
                params = {'reduce_only': True})
        quantity = round((float(balance['ETH']['free']) * .95), 3)
        order = client.create_order(
                symbol = 'ETHUSD',
                type = 'market',
                side = 'sell',
                amount = (price * quantity) * 5,
                params = {'leverage': 5})
        print(order)

In [ ]:
df.to_csv('check.csv', index=False)

In [ ]:
df.plot(y=['close', 'EMA20'], x='date', figsize=(20, 10))